# CZII making datasets for YOLO

This is a challenging competition in which participants must identify the location of particles contained in a 3D volumetric image.

There are already some great baselines published, but most of them focus on 3D volumetric images.

However, using 3D images directly is difficult: for example, we always have to be careful about VRAM consumption: even a small 3D image uses a lot of memory.

Therefore, I propose to decompose the 3D data provided by the host into 2D image slices and reduce it to an object detection problem.

This method allows us to treat just 7 3D images as more than 1k 2D images, mitigating the data scarcity problem.

# Install and Import modules

In [1]:
#!pip install zarr

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zarr
from tqdm import tqdm
import glob, os
import cv2

In [3]:
import os
import glob

# Get sorted list of run directories
runs = sorted(glob.glob(r"C:\Users\ifons\AI_Boot_Camp\project3\czii-cryo-et-object-identification\train\overlay\ExperimentRuns\*"))
runs = [os.path.basename(x) for x in runs]

# Create index-to-run and run-to-index dictionaries
i2r_dict = {i: r for i, r in zip(range(len(runs)), runs)}
r2i_dict = {r: i for i, r in zip(range(len(runs)), runs)}

# Print the dictionaries
i2r_dict
#print("Run to Index Dictionary:", r2i_dict)



{0: 'TS_5_4',
 1: 'TS_69_2',
 2: 'TS_6_4',
 3: 'TS_6_6',
 4: 'TS_73_6',
 5: 'TS_86_3',
 6: 'TS_99_9'}

# Normalize Function
To treat it as an image, normalize it to a value between 0 and 255.

1e-12 is very small and has the meaning of epsilon.

In [4]:
def convert_to_8bit(x):
    lower, upper = np.percentile(x, (0.5, 99.5))
    x = np.clip(x, lower, upper)
    x = (x - x.min()) / (x.max() - x.min() + 1e-12) * 255
    return x.round().astype("uint8")

# Information about labels

In [5]:
p2i_dict = {
        'apo-ferritin': 0,
        'beta-amylase': 1,
        'beta-galactosidase': 2,
        'ribosome': 3,
        'thyroglobulin': 4,
        'virus-like-particle': 5
    }

i2p = {v:k for k, v in p2i_dict.items()}

particle_radius = {
        'apo-ferritin': 60,
        'beta-amylase': 65,
        'beta-galactosidase': 90,
        'ribosome': 150,
        'thyroglobulin': 130,
        'virus-like-particle': 135,
    }

In [6]:
particle_names = ['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome', 'thyroglobulin', 'virus-like-particle']

# Main function for making datasets for YOLO
This is the main function.

Watch that YOLO annotation requires normalized 0 to 1 value range and (center_x, center_y, width, height) coordinate format.

In [7]:
def make_annotate_yolo(run_name, is_train_path=True):
    # to split validation
    is_train_path = 'train' if is_train_path else 'val'

    # read a volume
    vol = zarr.open(f'/kaggle/input/czii-cryo-et-object-identification/train/static/ExperimentRuns/{r}/VoxelSpacing10.000/denoised.zarr', mode='r')
    # use largest images
    vol = vol[0]
    # normalize [0, 255]
    vol2 = convert_to_8bit(vol)
    
    n_imgs = vol2.shape[0]
    # process each slices
    for j in range(n_imgs):
        newvol = vol2[j]
        newvolf = np.stack([newvol]*3, axis=-1)
        # YOLO requires image_size is multiple of 32
        newvolf = cv2.resize(newvolf, (640,640))
        # save as 1 slice
        cv2.imwrite(f'images/{is_train_path}/{run_name}_{j*10}.png', newvolf)
        # make txt file for annotation
        with open(f'labels/{is_train_path}/{run_name}_{j*10}.txt', 'w'):
            pass # make empty file
            
    # process each paticle types
    for p, particle in enumerate(tqdm(particle_names)):
        # we do not have to detect beta-amylase which weight is 0
        if particle=="beta-amylase":
            continue
        json_each_paticle = f"/kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/{run_name}/Picks/{particle}.json"
        df = pd.read_json(json_each_paticle) 
        # pick each coordinate of particles
        for axis in "x", "y", "z":
            df[axis] = df.points.apply(lambda x: x["location"][axis])

        
        radius = particle_radius[particle]
        for i, row in df.iterrows():
            # The radius from the center of the particle is used to determine the slices present.
            start_z = np.round(row['z'] - radius).astype(np.int32)
            start_z = max(0, start_z//10) # 10 means pixelspacing
            end_z = np.round(row['z'] + radius).astype(np.int32)
            end_z = min(n_imgs, end_z//10) # 10 means pixelspacing
            
            for j in range(start_z+1, end_z+1-1, 1):
                # white the results of annotation
                with open(f'labels/{is_train_path}/{run_name}_{j*10}.txt', 'a') as f:
                    f.write(f'{p2i_dict[particle]} {row["x"]/10/vol2.shape[1]} {row["y"]/10/vol2.shape[2]} {radius/10/vol2.shape[1]*2} {radius/10/vol2.shape[2]*2} \n')
    

In [8]:
# Assuming convert_to_8bit and particle_radius are already defined elsewhere in your code.

def make_annotate_yolo(run_name, is_train_path=True):
    # to split validation
    is_train_path = 'train' if is_train_path else 'val'

    # Construct the correct path
    path = f"C:\\Users\\ifons\\AI_Boot_Camp\\project3\\czii-cryo-et-object-identification\\train\\static\\ExperimentRuns\\{run_name}\\VoxelSpacing10.000\\denoised.zarr"
    
    # Check if the path exists
    if not os.path.exists(path):
        print(f"Path does not exist: {path}")
        return
    
    # read a volume
    vol = zarr.open(path, mode='r')
    # use largest images
    vol = vol[0]
    # normalize [0, 255]
    vol2 = convert_to_8bit(vol)
    
    n_imgs = vol2.shape[0]
    # process each slices
    for j in range(n_imgs):
        newvol = vol2[j]
        newvolf = np.stack([newvol]*3, axis=-1)
        # YOLO requires image_size is multiple of 32
        newvolf = cv2.resize(newvolf, (640, 640))
        # save as 1 slice
        cv2.imwrite(f'images/{is_train_path}/{run_name}_{j*10}.png', newvolf)
        # make txt file for annotation
        with open(f'labels/{is_train_path}/{run_name}_{j*10}.txt', 'w'):
            pass  # make empty file
            
    # process each particle type
    for p, particle in enumerate(tqdm(particle_names)):
        # we do not have to detect beta-amylase which weight is 0
        if particle == "beta-amylase":
            continue
        
        json_each_particle = f"C:\\Users\\ifons\\AI_Boot_Camp\\project3\\czii-cryo-et-object-identification\\train\\overlay\\ExperimentRuns\\{run_name}\\Picks\\{particle}.json"
        if not os.path.exists(json_each_particle):
            print(f"File does not exist: {json_each_particle}")
            continue
        
        df = pd.read_json(json_each_particle)
        # pick each coordinate of particles
        for axis in "x", "y", "z":
            df[axis] = df.points.apply(lambda x: x["location"][axis])

        radius = particle_radius[particle]
        for i, row in df.iterrows():
            # The radius from the center of the particle is used to determine the slices present.
            start_z = np.round(row['z'] - radius).astype(np.int32)
            start_z = max(0, start_z // 10)  # 10 means pixel spacing
            end_z = np.round(row['z'] + radius).astype(np.int32)
            end_z = min(n_imgs, end_z // 10)  # 10 means pixel spacing
            
            for j in range(start_z + 1, end_z + 1 - 1, 1):
                # write the results of annotation
                with open(f'labels/{is_train_path}/{run_name}_{j*10}.txt', 'a') as f:
                    f.write(f'{p2i_dict[particle]} {row["x"]/10/vol2.shape[1]} {row["y"]/10/vol2.shape[2]} {radius/10/vol2.shape[1]*2} {radius/10/vol2.shape[2]*2}\n')

# Example usage
runs = sorted(glob.glob(r"C:\Users\ifons\AI_Boot_Camp\project3\czii-cryo-et-object-identification\train\overlay\ExperimentRuns\*"))
runs = [os.path.basename(x) for x in runs]

# Use the first run as validation
for i, r in enumerate(runs):
    make_annotate_yolo(r, is_train_path=False if i == 0 else True)



100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.37it/s]


# Prepare Folders

In [9]:
os.makedirs("images/train", exist_ok=True)
os.makedirs("images/val", exist_ok=True)
os.makedirs("labels/val", exist_ok=True)
os.makedirs("labels/train", exist_ok=True)

# Main loop to make slice images and annotations

In [10]:
# use TS_5_4 as validation
for i, r in enumerate(runs):
    make_annotate_yolo(r, is_train_path=False if i==0 else True)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.48it/s]


Put them all in one folder.

In [11]:
import shutil
os.makedirs('datasets/czii_det2d', exist_ok=True)
shutil.move('images/train', 'datasets/czii_det2d/images/train')
shutil.move('images/val', 'datasets/czii_det2d/images')
shutil.move('labels/train', 'datasets/czii_det2d/labels/train')
shutil.move('labels/val', 'datasets/czii_det2d/labels')

'datasets/czii_det2d/labels\\val'

# make yaml file for Training 
We need to create a yaml configuration file for training, the format of which will not be detailed here.

In [12]:
%%writefile czii_conf.yaml

path: C:/Users/ifons/AI_Boot_Camp/project3/czii-cryo-et-object-identification/train/static/ExperimentRuns # dataset root dir
train: images/train # train images (relative to 'path') 
val: images/val # val images (relative to 'path') 

# Classes
names:
  0: apo-ferritin
  1: beta-amylase
  2: beta-galactosidase
  3: ribosome
  4: thyroglobulin
  5: virus-like-particle


Overwriting czii_conf.yaml


# Continue to Model czii_yolo11_training_2D_baseline.ipynb 